# Level 1 FEAT setup!

## Open up a fresh blank script in your Spyder IDE and save it as make_level1.py
For this tutorial you will input all code into this script and periodically commit your changes and push them to github when you see the logo 
![](https://imageog.flaticon.com/icons/png/512/3/3641.png?size=1200x630f&pad=10,10,10,10&ext=png&bg=FFFFFFFF)

## As always we are going to start by importing our modules

In [ ]:
import glob
import os
from subprocess import check_output
#import pdb
import argparse
from IPython.display import Image
from IPython.core.display import HTML 

## Lets start by building our two functions. This time we will call the first create_fsf( ) and as always main( )

```
def create_fsf():
```

```
def main ():
```

```
main()
```

![](https://imageog.flaticon.com/icons/png/512/3/3641.png?size=1200x630f&pad=10,10,10,10&ext=png&bg=FFFFFFFF)

## Strategy: 

We are going to use the .fsf file to automate our feat analysis. To do this we are going to need to do the following:
1. Decide on our analysis plan (what variables are we using? This should be done before we even try to automate)
2. Get a .fsf and find all the variables we are going to automate and which will be hard coded
3. Use a dictonary in python to fill in our fsf file
4. Make a script to run our new fsf files in parallel

## 1. Decide on our analysis plan (what variables are we using? This should be done before we even try to automate)

Normally this would be set out before you even gathered your data, or if you are doing secondary data analysis, before you dive into the data. You could even do a spiffy pre-reg https://osf.io/

Since this is a workshop we are going to pretend we did all that leg work. We are going to use the bart task and look at the difference between CASH trials and the ACCEPT trials and EXPLODE trials. Does this analysis make sense in real life? I don't know, but that doesn't really matter here.



## 2. Get a .fsf and find all the variables we are going to automate and which will be hard coded

The easiest way to do this is to fire up the Feat gui and to start a list of variables we are going to hard code and those we are going to fill in through python.

![alt text](https://github.com/grace-shearrer/nibl_workshop/blob/master/feat_data.png?raw=true "Logo Title Text 1")

We can do a lot of heavy lifting here. 
Notice my output directory is now just OUTPUT
I have variables for the total volumes and the TRs
I try to populate as many fields as possible with info from the image 

![](https://github.com/grace-shearrer/nibl_workshop/blob/master/feat_input.png?raw=true)

![](https://github.com/grace-shearrer/nibl_workshop/blob/master/feat_prestats.png?raw=true)

Alternatively, the pre-stats can almost be completely filled into the GUI once
Why would we ever change this between runs/subjects? I can't think of a reason

## Now to the registration tab, this is big choice point! Are we planning on using data registered elsewhere?

## Nope gonna register it right here

![](https://github.com/grace-shearrer/nibl_workshop/blob/master/feat_reg.png?raw=true)

Since we are loading in our anatomical image we are going to create a variable for it. Make sure you have the correct path to your standard space!

## I'll hand my own registration elsewhere thank you

![](https://github.com/grace-shearrer/nibl_workshop/blob/master/feat_noreg.png?raw=true)

We probably spent a long time (or computing power) on our custom registration, so let's save time here and use only 6 DOF ;)

## The heavy lifting: The Stats Tab!

![](https://github.com/grace-shearrer/nibl_workshop/blob/master/feat_stats.png?raw=true)

Notice we have both the FILM prewhiting tab and the Add additional confounds checked.

![](https://github.com/grace-shearrer/nibl_workshop/blob/master/feat_conf.png?raw=true)

![](https://github.com/grace-shearrer/nibl_workshop/blob/master/feat_modelev.png?raw=true)

Here we can set a variable for each EVTITLE and the EV itself
We just need to make sure they match 
We can also do this for the motion correction parameters (if we chose not to do it through FEAT)

![](https://github.com/grace-shearrer/nibl_workshop/blob/master/feat_moco.png?raw=true)

Notice for the motion correction I have set the shape to a single column and the convolution to none.
#### Concept Check: Why did set the convolution to none?

![](https://github.com/grace-shearrer/nibl_workshop/blob/master/feat_modelcon.png?raw=true)

Here are the fake contrasts we will look at
I like to take a screenshot of this as a reference since after this fsl will refer to these by number only
#### Future Knowledge: What does fsl call the higher level directories based on our contrasts?

## Last but not least (ok kinda least): POST STATS

![](https://github.com/grace-shearrer/nibl_workshop/blob/master/feat_poststats.png?raw=true)

My personal favorite, turn everything off we will come back to this when we do our group level model

## Great we are done with our one GUI, and if you tend to do similar analyeses maybe the last ever! Click save and save the file as:
## Level1_design.fsf
The GUI will get mad at you and tell you you are missing a lot of stuff. Just click through.
Make sure the .fsf file was generated. We can delete all the other files.

### Let's remind ourselves what we variables we will need to generate
1. FUNCRUN = this will be our input image
2. NTIMEPOINTS = this is the number of timepoints per image
3. TRS = this is the TRs in the nifti
4. OUTPUT = what we want to call our output directory, it will be something .feat
5. ANAT = this is our T1W image, if we are using FNIRT it needs have skull, if we are using FLIRT we want the defaced (betted)
6. CONFOUNDS = this is the confound.txt file generated from fsl_motion_outliers that we should have in our motion assessment directory if our prepro script worked
7. EV = this is the path to the explanatory variable text file we want to use (we will have more than 1), each needs to be in the 3 column format
8. EVTITLE = this is a useful name for each EV
9. MOTCOR = this is the path to the motion parameters we generated in with the preprocessing script

### Let's also remember a major choice point
Are we using feat to register or no?

## At this time let's open that .fsf file we just created with our text editor and take a look at our handy work!

![](https://imageog.flaticon.com/icons/png/512/3/3641.png?size=1200x630f&pad=10,10,10,10&ext=png&bg=FFFFFFFF)

## Okay now for the actual python part! 
right now our script looks a little bare

```
def create_fsf():
    
def main():
    
main()
```

## Let's set up some globals
### What are we sure we are going to need?
1. basedir = this is going to be like second nature, where is our data?
2. outdir = where is our output data located, if we are using a bids like structure probably some where with derivative in the directory tree
3. Do we want to pass arguments via argparse? YES 
   * We have multiple tasks so we will want something argument to differentiate task
   * We have a lot of potential EVs so we will want something that will take multiple arguments
   * We have a serious choice point of to reg or not to reg (shakespeare's got nothing on us)
### Give it a try and write your main( ) function

In [ ]:
def main ():
  basedir='/Users/gracer/Desktop/data'
  outdir=os.path.join(basedir,'derivatives','task')
  parser=argparse.ArgumentParser(description='making fsf files')
  parser.add_argument('-noreg',dest='NOREG', action='store_true',
                        default=False, help='Did you already register your data (using ANTZ maybe)?')
  parser.add_argument('-task',dest='TASK',
                        default=False, help='which task are we using?')
  parser.add_argument('-evs',dest='EV',nargs='+',
                        default=False, help='which evs are we using?')

  repl_dict={}
  args = parser.parse_args()
  arglist={}
  for a in args._get_kwargs():
      arglist[a[0]]=a[1]
      print(arglist)
  create_fsf(basedir,repl_dict, outdir, arglist)

## This is what I came up with does anyone have anything different? There are a lot of ways we could accomplish this

![](https://imageog.flaticon.com/icons/png/512/3/3641.png?size=1200x630f&pad=10,10,10,10&ext=png&bg=FFFFFFFF)

## On to the meaty bits! The create_fsf( ) function!

## We need to get all the subjects.
### Concept Check: What can we use to get all the subjects?

![](https://i.imgur.com/itVtNcK.gif)

In [ ]:
basedir='/Users/gracer/Desktop/data'
listy=glob.glob(os.path.join(basedir,'sub-*','func','*.nii.gz'))
print(listy[1].split('/'))
repl_dict={}

## 1. funcrun

In [ ]:
def create_fsf():
    for item in glob.glob(os.path.join(basedir,'sub-*')):
        sub=item.split('/')[5]
        funcrun=(os.path.join(item,'func','%s_task-%s_bold_brain_mcf.nii.gz')%(sub,arglist['TASK']))
        repl_dict.update({'FUNCRUN':funcrun})

## 2. NTIMEPOINTS = this is the number of timepoints per image
## 3. TRS = this is the TRs in the nifti

In [ ]:
def create_fsf():
    for item in glob.glob(os.path.join(basedir,'sub-*')):
        sub=item.split('/')[5]
        funcrun=(os.path.join(item,'func','%s_task-%s_bold_brain_mcf.nii.gz')%(sub,arglist['TASK']))
        repl_dict.update({'FUNCRUN':funcrun})
        
        ntmpts=check_output(['fslnvols',funcrun])
        repl_dict.update({'NTIMEPOINTS':ntmpts})

        trs=check_output(['fslval','%s'%(funcrun),'pixdim4',scan])
        print(trs)
        repl_dict.update({'TRS':trs})

## 4. OUTPUT = what we want to call our output directory, it will be something .feat
## 5. ANAT = this is our T1W image, if we are using FNIRT it needs have skull, if we are using FLIRT we want the defaced (betted)

In [ ]:
        output=os.path.join(outdir,sub,'grace_edit',arglist['TASK'])
        repl_dict.update({'OUTPUT':output})            
        anat=os.path.join(basedir,sub,'anat','%s_T1w_brain.nii.gz'%(sub))
        repl_dict.update({'ANAT':anat})

## 6. CONFOUNDS = this is the confound.txt file generated from fsl_motion_outliers that we should have in our motion assessment directory if our prepro script worked

In [ ]:
        confounds=os.path.join(basedir,sub,'func','motion_assessment','%s_task-%s_bold_brain_confound.txt'%(sub,arglist['TASK']))
        repl_dict.update({'CONFOUNDS':confounds})

![](https://imageog.flaticon.com/icons/png/512/3/3641.png?size=1200x630f&pad=10,10,10,10&ext=png&bg=FFFFFFFF)

## 7. EV = this is the path to the explanatory variable text file we want to use (we will have more than 1), each needs to be in the 3 column format
## 8. EVTITLE = this is a useful name for each EV

This is going to be a tough one. We have a certain number of EVs we have defined using our argparse. How are we going to match it up with the EVs and EV titles? 
Let's take a look at what the EV files look like.

#### sub-10171_bart_action_EXPLODE_output.txt

What parts of this do we need to be able to swap out?
1. subject
2. task
3. action type (EXPLODE, CASHOUT, ACCEPT)

### Concept Check: Where have we already defined these? Or do we need to define them now?

1. subject = we have defined this as sub
2. task = we have this in our arglist dictionary 
3. action type (EXPLODE, CASHOUT, ACCEPT) = we have this in our arglist dictionary 

We can use a counter to loop through our EVs. 
This gives us the numbers we will need in the EV title.

In [ ]:
        ctr=0
        for item in arglist['EV']:
            print(item)
            ctr=ctr+1
            repl_dict.update({'EV%iTITLE'%ctr:item})
            ev=os.path.join(basedir,sub,'func','onsets','%s_%s_%s_output.txt'%(sub,arglist['TASK'],item))
            repl_dict.update({'EV%i'%ctr:ev}) 

## 9. MOTCOR = this is the path to the motion parameters we generated in with the preprocessing script

Similar to the EVs we will need to use a counter for the motion parameters. 
Unlike the EVs though, we will always only have 6 motion parameters.
We can use the range function as a counter!

In [ ]:
    for i in range(6):
        motcor=os.path.join(basedir,sub,'func','motion_assessment','%s_task-%s_bold_brain_motcor%i.txt' %(sub,arglist['TASK'],i))
        repl_dict.update({'MOTCOR%i'%i:motcor})

![](https://imageog.flaticon.com/icons/png/512/3/3641.png?size=1200x630f&pad=10,10,10,10&ext=png&bg=FFFFFFFF)

## Let's add a print statement incase we need to debug

In [8]:
print(repl_dict)

NameError: name 'repl_dict' is not defined

## We should now have something that looks like this

In [ ]:
def create_fsf(basedir,repl_dict,outdir, arglist):
    os.chdir(basedir)
    for sub in glob.glob('sub-*'):
        repl_dict.update({'SUB':sub})

        scan=(os.path.join(sub,'func','%s_task-%s_bold_brain_mcf.nii.gz')%(sub,arglist['TASK']))

        funcrun=os.path.join(basedir,scan)
        repl_dict.update({'FUNCRUN':funcrun})

        ntmpts=check_output(['fslnvols',funcrun])
        repl_dict.update({'NTIMEPOINTS':ntmpts})

        trs=check_output(['fslval','%s'%(funcrun),'pixdim4',scan])
        print(trs)
        repl_dict.update({'TRS':trs})


        output=os.path.join(outdir,sub,'grace_edit',arglist['TASK'])
        repl_dict.update({'OUTPUT':output})            
        anat=os.path.join(basedir,sub,'anat','%s_T1w_brain.nii.gz'%(sub))
        repl_dict.update({'ANAT':anat})

        confounds=os.path.join(basedir,sub,'func','motion_assessment','%s_task-%s_bold_brain_confound.txt'%(sub,arglist['TASK']))
        repl_dict.update({'CONFOUNDS':confounds})
        ctr=0
        for item in arglist['EV']:
            print(item)
            ctr=ctr+1
            repl_dict.update({'EV%iTITLE'%ctr:item})
            ev=os.path.join(basedir,sub,'func','onsets','%s_%s_%s_output.txt'%(sub,arglist['TASK'],item))
            repl_dict.update({'EV%i'%ctr:ev}) 
        for i in range(6):
            motcor=os.path.join(basedir,sub,'func','motion_assessment','%s_task-%s_bold_brain_motcor%i.txt' %(sub,arglist['TASK'],i))
            repl_dict.update({'MOTCOR%i'%i:motcor})

        print(repl_dict)

## We have all our variables now we need to swap out the place holder variables in our .fsf files

### First we need to open a file to read
```
with open(os.path.join(basedir,'design.fsf'),'r') as infile:
```
### Second we need to read the file.
You may recall we previously have used the function readlines(), this time we don't want to read in the file as a list.
We want to read in the file as a string.
```
tempfsf=infile.read()
```
### Third we need to loop through our dictionary using the key
```
for key in repl_dict:
    tempfsf = tempfsf.replace(key, repl_dict[key])
```
We can use the replace function to replace the variables in the fsf file with the values in the dictionary.

### Concept Check: the replace function requires a string argument. If we decided to use the readlines function instead of the read function, would replace still work?

### Now that we have replaced the variables in the fsf with the key values we can write the strings to a new file 
```
with open(os.path.join(outdir,sub,'%s_%s.fsf'%(sub,arglist['TASK'])),'w') as outfile:
    outfile.write(tempfsf)
outfile.close()
```

### All together we should have this

In [ ]:
def create_fsf(basedir,repl_dict,outdir, arglist):
    os.chdir(basedir)
    for sub in glob.glob('sub-*'):
        repl_dict.update({'SUB':sub})

        scan=(os.path.join(sub,'func','%s_task-%s_bold_brain_mcf.nii.gz')%(sub,arglist['TASK']))

        funcrun=os.path.join(basedir,scan)
        repl_dict.update({'FUNCRUN':funcrun})

        ntmpts=check_output(['fslnvols',funcrun])
        repl_dict.update({'NTIMEPOINTS':ntmpts})

        trs=check_output(['fslval','%s'%(funcrun),'pixdim4',scan])
        print(trs)
        repl_dict.update({'TRS':trs})


        output=os.path.join(outdir,sub,'grace_edit',arglist['TASK'])
        repl_dict.update({'OUTPUT':output})            
        anat=os.path.join(basedir,sub,'anat','%s_T1w_brain.nii.gz'%(sub))
        repl_dict.update({'ANAT':anat})

        confounds=os.path.join(basedir,sub,'func','motion_assessment','%s_task-%s_bold_brain_confound.txt'%(sub,arglist['TASK']))
        repl_dict.update({'CONFOUNDS':confounds})
        ctr=0
        for item in arglist['EV']:
            print(item)
            ctr=ctr+1
            repl_dict.update({'EV%iTITLE'%ctr:item})
            ev=os.path.join(basedir,sub,'func','onsets','%s_%s_%s_output.txt'%(sub,arglist['TASK'],item))
            repl_dict.update({'EV%i'%ctr:ev}) 
        for i in range(6):
            motcor=os.path.join(basedir,sub,'func','motion_assessment','%s_task-%s_bold_brain_motcor%i.txt' %(sub,arglist['TASK'],i))
            repl_dict.update({'MOTCOR%i'%i:motcor})

        print(repl_dict)
        if arglist['NOREG']==False:
            with open(os.path.join(basedir,'design.fsf'),'r') as infile:
                tempfsf=infile.read()
                for key in repl_dict:
                    tempfsf = tempfsf.replace(key, repl_dict[key])
                    with open(os.path.join(outdir,sub,'%s_%s.fsf'%(sub,arglist['TASK'])),'w') as outfile:
                        outfile.write(tempfsf)
                    outfile.close()
            infile.close()

![](https://imageog.flaticon.com/icons/png/512/3/3641.png?size=1200x630f&pad=10,10,10,10&ext=png&bg=FFFFFFFF)

## But wait... what about the registration issue???
### To address this we can use an if/else statement with our arglist
### Give it a try first

In [ ]:
if arglist['NOREG']==False:
            with open(os.path.join(basedir,'design.fsf'),'r') as infile:
                tempfsf=infile.read()
                for key in repl_dict:
                    tempfsf = tempfsf.replace(key, repl_dict[key])
                    with open(os.path.join(outdir,sub,'%s_%s.fsf'%(sub,arglist['TASK'])),'w') as outfile:
                        outfile.write(tempfsf)
                    outfile.close()
                infile.close()

        else:
            print("skipping registration")
            with open(os.path.join(basedir,'no_reg_design.fsf'),'r') as infile:
                tempfsf=infile.read()
                for key in repl_dict:
                    tempfsf = tempfsf.replace(key, repl_dict[key])
                    with open(os.path.join(outdir,sub,'%s_%s_no_reg.fsf'%(sub,arglist['TASK'])),'w') as outfile:
                        outfile.write(tempfsf)
                    outfile.close()
                infile.close()

## All together now!

In [ ]:
#!/usr/bin/env python

import glob
import os
from subprocess import check_output
#import pdb
import argparse


def create_fsf(basedir,repl_dict,outdir, arglist):
    os.chdir(basedir)
    for sub in glob.glob('sub-*'):
        repl_dict.update({'SUB':sub})

        scan=(os.path.join(sub,'func','%s_task-%s_bold_brain_mcf.nii.gz')%(sub,arglist['TASK']))

        funcrun=os.path.join(basedir,scan)
        repl_dict.update({'FUNCRUN':funcrun})

        ntmpts=check_output(['fslnvols',funcrun])
        repl_dict.update({'NTIMEPOINTS':ntmpts})

        trs=check_output(['fslval','%s'%(funcrun),'pixdim4',scan])
        print(trs)
        repl_dict.update({'TRS':trs})


        output=os.path.join(outdir,sub,'grace_edit',arglist['TASK'])
        repl_dict.update({'OUTPUT':output})            
        anat=os.path.join(basedir,sub,'anat','%s_T1w_brain.nii.gz'%(sub))
        repl_dict.update({'ANAT':anat})

        confounds=os.path.join(basedir,sub,'func','motion_assessment','%s_task-%s_bold_brain_confound.txt'%(sub,arglist['TASK']))
        repl_dict.update({'CONFOUNDS':confounds})
        ctr=0
        for item in arglist['EV']:
            print(item)
            ctr=ctr+1
            repl_dict.update({'EV%iTITLE'%ctr:item})
            ev=os.path.join(basedir,sub,'func','onsets','%s_%s_%s_output.txt'%(sub,arglist['TASK'],item))
            repl_dict.update({'EV%i'%ctr:ev}) 
        for i in range(6):
            motcor=os.path.join(basedir,sub,'func','motion_assessment','%s_task-%s_bold_brain_motcor%i.txt' %(sub,arglist['TASK'],i))
            repl_dict.update({'MOTCOR%i'%i:motcor})

        print(repl_dict)
        if arglist['NOREG']==False:
            with open(os.path.join(basedir,'design.fsf'),'r') as infile:
                tempfsf=infile.read()
                for key in repl_dict:
                    tempfsf = tempfsf.replace(key, repl_dict[key])
                    with open(os.path.join(outdir,sub,'%s_%s.fsf'%(sub,arglist['TASK'])),'w') as outfile:
                        outfile.write(tempfsf)
                    outfile.close()
            infile.close()

        else:
            print("skipping registration")
            with open(os.path.join(basedir,'no_reg_design.fsf'),'r') as infile:
                tempfsf=infile.read()
                for key in repl_dict:
                    tempfsf = tempfsf.replace(key, repl_dict[key])
                    with open(os.path.join(outdir,sub,'%s_%s_no_reg.fsf'%(sub,arglist['TASK'])),'w') as outfile:
                        outfile.write(tempfsf)
                    outfile.close()
            infile.close()
    os.chdir('/Users/gracer/Google Drive/fMRI_workshop/scripts/feat_scripts')
                       

def main ():
  basedir='/Users/gracer/Desktop/data'
  outdir=os.path.join(basedir,'derivatives','task')
  parser=argparse.ArgumentParser(description='making fsf files')
  parser.add_argument('-noreg',dest='NOREG', action='store_true',
                        default=False, help='Did you already register your data (using ANTZ maybe)?')
  parser.add_argument('-task',dest='TASK',
                        default=False, help='which task are we using?')
  parser.add_argument('-evs',dest='EV',nargs='+',
                        default=False, help='which evs are we using?')

  repl_dict={}
  args = parser.parse_args()
  arglist={}
  for a in args._get_kwargs():
      arglist[a[0]]=a[1]
      print(arglist)
  create_fsf(basedir,repl_dict, outdir, arglist)
main()
os.chdir('/Users/gracer/Google Drive/fMRI_workshop/scripts/feat_scripts')